# Offline Step: Generation of Training Artifacts

In [ ]:
%pip install --upgrade pip
%pip install torch-ort torchvision

In [ ]:
# largely borrowed from https://www.youtube.com/watch?v=u7YCaiHOC9o

import torch
import os
import torchvision

os.makedirs("training_artifacts", exist_ok=True)

model = torchvision.models.mobilenet_v2(
    weights=torchvision.models.MobileNet_V2_Weights.IMAGENET1K_V2
)


# imagenet had 1000 classes, bloodMNIST only has 8
model.classifier[1] = torch.nn.Linear(1280, 8)

# convert the model to ONNX
model_name = "mobilenetv2"
torch.onnx.export(model, torch.randn(1, 3, 224, 224),
                  f"training_artifacts/{model_name}.onnx",
                  input_names=["input"],
                  output_names=["output"],
                  dynamic_axes={"input": {0: "batch"}, "output": {0: "batch"}},
                  export_params=True,
                  do_constant_folding=False,
                  training=torch.onnx.TrainingMode.TRAINING)

Now we have a model converted to ONNX format, need to generate the training artifacts that we will use later:

In [ ]:
import onnx
from onnxruntime.training import artifacts

# get the onnx model
onnx_model = onnx.load(f"training_artifacts/{model_name}.onnx")